# example 1: provide data and use existing model

In [1]:
!textattack attack --recipe textfooler --dataset-from-file "test_data.py" --model bert-base-uncased-mr

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
textattack: Loading pre-trained model from HuggingFace model repository: textattack/bert-base-uncased-rotten-tomatoes
Using /tmp/tfhub_modules to cache modules.
textattack: Goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'> compatible with model BertForSequenceClassification.
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding_type):  paragramcf
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding_type):  paragramcf
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
      )
    (2): Univers

# example2(not working yet):provide model

In [1]:
!textattack attack --model-from-file model1.py --recipe deepwordbug --num-examples 1000

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
textattack: Loading model and tokenizer from file: model1.py
textattack: Loading module from `model1.py`.
Traceback (most recent call last):
  File "/home/qc/anaconda3/lib/python3.7/site-packages/textattack/commands/attack/attack_args_helpers.py", line 252, in parse_model_from_args
    model_module = load_module_from_file(args.model_from_file)
  File "/home/qc/anaconda3/lib/python3.7/site-packages/textattack/commands/attack/attack_args_helpers.py", line 108, in load_module_from_file
    spec.loader.exec_module(module)
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "model1.py", line 23, in <module>
    tokenizer = Tokenizer()
  File "model1.py", line 15, in __init__
    self.t = pd.read_pickle("./tf_idf_vectorizor.pkl")
  File "/home/qc/anaconda3/lib/python3.7/site-packag

# example 3: (from doc)NLTK and Named Entity Recognition

In [2]:
import nltk
nltk.download('punkt') # The NLTK tokenizer
nltk.download('maxent_ne_chunker') # NLTK named-entity chunker
nltk.download('words') # NLTK list of words

[nltk_data] Downloading package punkt to /home/qc/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/qc/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /home/qc/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [3]:
sentence = ('In 2017, star quarterback Tom Brady led the Patriots to the Super Bowl, '
           'but lost to the Philadelphia Eagles.')

# 1. Tokenize using the NLTK tokenizer.
tokens = nltk.word_tokenize(sentence)

# 2. Tag parts of speech using the NLTK part-of-speech tagger.
tagged = nltk.pos_tag(tokens)

# 3. Extract entities from tagged sentence.
entities = nltk.chunk.ne_chunk(tagged)
print(entities)

(S
  In/IN
  2017/CD
  ,/,
  star/NN
  quarterback/NN
  (PERSON Tom/NNP Brady/NNP)
  led/VBD
  the/DT
  (ORGANIZATION Patriots/NNP)
  to/TO
  the/DT
  (ORGANIZATION Super/NNP Bowl/NNP)
  ,/,
  but/CC
  lost/VBD
  to/TO
  the/DT
  (ORGANIZATION Philadelphia/NNP Eagles/NNP)
  ./.)


In [4]:
# 4. Filter entities to just named entities.
named_entities = [entity for entity in entities if isinstance(entity, nltk.tree.Tree)]
print(named_entities)

[Tree('PERSON', [('Tom', 'NNP'), ('Brady', 'NNP')]), Tree('ORGANIZATION', [('Patriots', 'NNP')]), Tree('ORGANIZATION', [('Super', 'NNP'), ('Bowl', 'NNP')]), Tree('ORGANIZATION', [('Philadelphia', 'NNP'), ('Eagles', 'NNP')])]


In [5]:
import functools

@functools.lru_cache(maxsize=2**14)
def get_entities(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)
    # Setting `binary=True` makes NLTK return all of the named
    # entities tagged as NNP instead of detailed tags like
    #'Organization', 'Geo-Political Entity', etc.
    entities = nltk.chunk.ne_chunk(tagged, binary=True)
    return entities.leaves()

In [6]:
sentence = 'Jack Black starred in the 2003 film classic "School of Rock".'
get_entities(sentence)

[('Jack', 'NNP'),
 ('Black', 'NNP'),
 ('starred', 'VBD'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('2003', 'CD'),
 ('film', 'NN'),
 ('classic', 'JJ'),
 ('``', '``'),
 ('School', 'NNP'),
 ('of', 'IN'),
 ('Rock', 'NNP'),
 ("''", "''"),
 ('.', '.')]

In [7]:
from textattack.constraints import Constraint

class NamedEntityConstraint(Constraint):
    """ A constraint that ensures `transformed_text` only substitutes named entities from `current_text` with other named entities.
    """
    def _check_constraint(self, tranformed_text, current_text, original_text=None):
        transformed_entities = get_entities(transformed_text.text)
        current_entities = get_entities(current_text.text)
        # If there aren't named entities, let's return False (the attack
        # will eventually fail).
        if len(current_entities) == 0:
            return False
        if len(current_entities) != len(transformed_entities):
            # If the two sentences have a different number of entities, then 
            # they definitely don't have the same labels. In this case, the 
            # constraint is violated, and we return False.
            return False
        else:
            # Here we compare all of the words, in order, to make sure that they match.
            # If we find two words that don't match, this means a word was swapped 
            # between `current_text` and `transformed_text`. That word must be a named entity to fulfill our
            # constraint.
            current_word_label = None
            transformed_word_label = None
            for (word_1, label_1), (word_2, label_2) in zip(current_entities, transformed_entities):
                if word_1 != word_2:
                    # Finally, make sure that words swapped between `x` and `x_adv` are named entities. If 
                    # they're not, then we also return False.
                    if (label_1 not in ['NNP', 'NE']) or (label_2 not in ['NNP', 'NE']):
                        return False            
            # If we get here, all of the labels match up. Return True!
            return True

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [9]:
# Import the dataset.
from textattack.datasets.classification import YelpSentiment
# Create the model.
from textattack.models.classification.lstm import LSTMForYelpSentimentClassification
model = LSTMForYelpSentimentClassification()
# Create the goal function using the model.
from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model)

ModuleNotFoundError: No module named 'textattack.datasets.classification'

### cannot find the imported items in github.

# example 4: follow doc example to attack a sklearn model

In [1]:
import pandas as pd

logistic_reg = pd.read_pickle('./tf_idf_logistic_reg.pkl')

def spacy_tokenizer(sentence):
    import spacy
    nlp = spacy.load('en_core_web_md')
    mytokens = nlp(sentence)
    mytokens = [word.lemma_ for word in mytokens if not word.is_punct and not word.is_stop]
    return mytokens

t = pd.read_pickle('./tf_idf_vectorizor.pkl')

class Model:
    def __init__(self, t, m):
        self.tokenizer = t
        self.tokenizer.encode = self.tokenizer.transform
        self.model = m

In [2]:
from textattack.goal_functions import UntargetedClassification
model = Model(t, logistic_reg)
goal_function = UntargetedClassification(model)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
textattack: Unknown if model of class <class '__main__.Model'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [3]:
from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedySearch
from textattack.shared import Attack
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification

# We're going to the `WordSwapEmbedding` transformation. Using the default settings, this
# will try substituting words with their neighbors in the counter-fitted embedding space. 
transformation = WordSwapEmbedding(max_candidates=15) 

# We'll use the greedy search method again
search_method = GreedySearch()

# Our constraints will be the same as Tutorial 1, plus the named entity constraint
constraints = [RepeatModification(),
               StopwordModification(),
               #NamedEntityConstraint()
              ]

# Now, let's make the attack using these parameters. 
attack = Attack(goal_function, constraints, transformation, search_method)

print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding_type):  paragramcf
  )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [4]:
import thinc.extra.datasets
train, test = thinc.extra.datasets.imdb()

In [5]:
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult

results_iterable = attack.attack_dataset(train[:20]) #, attack_n=True) this argument does not exist
logger = CSVLogger(color_method='html')

num_successes = 0
while num_successes < 10:
    result = next(results_iterable)
    if isinstance(result, SuccessfulAttackResult):
        logger.log_attack_result(result)
        num_successes += 1

textattack: Failed to predict with model <class '__main__.Model'>. Check tokenizer configuration.


AttributeError: 'LogisticRegression' object has no attribute 'parameters'

# example 5: follow doc example to attack a pytorch model

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
from tensorboardX import SummaryWriter
import random

In [9]:
class ImdbRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers,
              is_bidirectional=False, dropout=0.0, output_dim=1, padding_idx=None):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, 
                                      padding_idx=padding_idx)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, 
                            num_layers=n_layers, bidirectional=is_bidirectional,
                           dropout=dropout)

        self.fc = nn.Linear((is_bidirectional+1)*hidden_dim, output_dim)

        self.is_bidirectional = is_bidirectional
    
    def forward(self, input_sequence, sequence_length):
    
        embeddings = self.embedding(input_sequence)

        packed_embeddings = nn.utils.rnn.pack_padded_sequence(embeddings, 
                                                            sequence_length)

        packed_output, (hidden_state, cell_state) = self.lstm(packed_embeddings)

        if self.is_bidirectional:
            output = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)
        else:
            output = hidden_state[-1,:,:]
        scores = self.fc(output)

        return scores

In [11]:
TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_md', lower=True, include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [12]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split()

In [13]:
MAX_VOCAB_SIZE = 25000
TEXT.build_vocab(train_data, 
                 max_size=MAX_VOCAB_SIZE,
                 vectors='glove.6B.300d',
                 unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [15]:
vocab_size = len(TEXT.vocab)
embedding_dim = 300 # This needs to match the size of the pre-trained embeddings!
hidden_dim = 256
num_layers = 3
dropout = 0.5
pad_idx = TEXT.vocab.stoi[TEXT.pad_token]
model = ImdbRNN(vocab_size=vocab_size, embedding_dim=embedding_dim,hidden_dim=hidden_dim,
                  n_layers=num_layers,  dropout=dropout,
                  padding_idx=pad_idx)

In [16]:
model.load_state_dict(torch.load('./state_dict.pt'))

<All keys matched successfully>

In [17]:
model = model.to(device)
model.eval()

ImdbRNN(
  (embedding): Embedding(25002, 300, padding_idx=1)
  (lstm): LSTM(300, 256, num_layers=3, dropout=0.5)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [18]:
from textattack.models.helpers.lstm_for_classification import LSTMForClassification

In [ ]:
m = LSTMForClassification()